In [ ]:
! pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.72
    Uninstalling opencv-python-4.7.0.72:
      Successfully uninstalled opencv-python-4.7.0.72


In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from loader_util.nn.conv import Resnet
from loader_util.callbacks import EpochCheckpoint
from loader_util.callbacks import TrainingMonitor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import numpy as np
import argparse
import sys

In [ ]:
# script constants
checkpoints_dir = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter12_resnet/outputs/models/cifar10"
model_to_load = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter12_resnet/outputs/models/cifar10/model_epoch_75.h5"
start_epoch = 75

In [ ]:
# load the training and testing data, converting the images from
# integers to floats
print("[INFO] loading CIFAR-10 data...")
((trainx, trainy), (testx, testy)) = cifar10.load_data()
trainx = trainx.astype("float")
testx = testx.astype("float")

# apply mean subtraction to the data
mean = np.mean(trainx, axis=0)
trainx -= mean
testx -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)
len(trainx)

[INFO] loading CIFAR-10 data...


50000

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# if there is no specific model checkpoint supplied, then initialize
# the network (ResNet-56) and compile the model
if not model_to_load:
    print("[INFO] compiling model...")
    opt = SGD(learning_rate=1e-1)
    model = Resnet.build(32, 32, 3,
                         classes=10,
                         stages=(9, 9, 9),
                         filters=(16, 16, 32, 64),
                         reg=0.0001)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model_to_load))
    model = load_model(model_to_load)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(
        K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-3)
    print("[INFO] new learning rate: {}".format(
        K.get_value(model.optimizer.lr)))

[INFO] loading /content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter12_resnet/outputs/models/cifar10/model_epoch_75.h5...
[INFO] old learning rate: 0.009999999776482582
[INFO] new learning rate: 0.0010000000474974513


In [ ]:
# construct the set of callbacks
import os
file_output_dir = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter12_resnet/outputs/files/cifar10"
callbacks = [
    EpochCheckpoint(checkpoints_dir,
                    every=5,
                    startAt=start_epoch),
    TrainingMonitor(figPath=os.path.sep.join([file_output_dir, "resnet56_cifar10.png"]),
                    jsonPath=os.path.sep.join([file_output_dir, "resnet56_cifar10.json"]),
                    startAt=start_epoch)]

In [ ]:
# train the network
print("[INFO] training network...")
model.fit_generator(
    aug.flow(trainx, trainy, batch_size=128),
    validation_data=(testx, testy),
    steps_per_epoch=len(trainx) // 128, epochs=10,
    callbacks=callbacks, verbose=1)

[INFO] training network...
Epoch 1/10


<ipython-input-30-8ffce5a256d1>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


390/390 [==============================] - 29s 74ms/step - loss: 0.4602 - accuracy: 0.8771 - val_loss: 0.5846 - val_accuracy: 0.8387
Epoch 2/10
390/390 [==============================] - 28s 72ms/step - loss: 0.4617 - accuracy: 0.8774 - val_loss: 0.5844 - val_accuracy: 0.8404
Epoch 3/10
390/390 [==============================] - 28s 72ms/step - loss: 0.4638 - accuracy: 0.8740 - val_loss: 0.5752 - val_accuracy: 0.8428
Epoch 4/10
390/390 [==============================] - 28s 72ms/step - loss: 0.4630 - accuracy: 0.8760 - val_loss: 0.5891 - val_accuracy: 0.8386
Epoch 5/10
390/390 [==============================] - 29s 74ms/step - loss: 0.4627 - accuracy: 0.8760 - val_loss: 0.5815 - val_accuracy: 0.8408
Epoch 6/10
390/390 [==============================] - 29s 74ms/step - loss: 0.4625 - accuracy: 0.8754 - val_loss: 0.5895 - val_accuracy: 0.8386
Epoch 7/10
390/390 [==============================] - 28s 72ms/step - loss: 0.4638 - accuracy: 0.8743 - val_loss: 0.5849 - val_accuracy: 0.8391
Epo